In [33]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest, Strategy

In [28]:
df = pd.read_csv('data/spgi.csv')
df['Date'] = pd.to_datetime(df['Date'], utc=True)
df.set_index('Date', inplace=True)
df=df[df.High!=df.Low]

In [45]:
df.ta.macd(fast=12, slow=26, signal=9, append=True)
df['Signal'] = np.where(df['MACDh_12_26_9'] > 0, 'B', np.where(df['MACDh_12_26_9'] < 0, 'S', 'N'))
df['ATR'] = ta.atr(df.High, df.Low, df.Close, length=14)
df

Open        High         Low       Close  \
Date                                                                        
2023-09-18 04:00:00+00:00  385.329232  389.561212  384.691932  387.868408   
2023-09-19 04:00:00+00:00  386.324987  388.216925  385.000640  387.659302   
2023-09-20 04:00:00+00:00  388.366297  390.457399  384.184123  384.602325   
2023-09-21 04:00:00+00:00  382.013362  383.367580  370.582026  370.731384   
2023-09-22 04:00:00+00:00  370.990287  374.306155  369.496647  370.114014   
...                               ...         ...         ...         ...   
2024-03-11 04:00:00+00:00  427.000000  428.690002  423.290009  426.679993   
2024-03-12 04:00:00+00:00  429.320007  430.730011  426.149994  428.609985   
2024-03-13 04:00:00+00:00  428.290009  430.170013  426.679993  428.029999   
2024-03-14 04:00:00+00:00  426.720001  428.100006  421.059998  423.470001   
2024-03-15 04:00:00+00:00  420.029999  424.519989  418.649994  422.809998   

                            Volume  Dividends  Stock Splits  MACD_12_26_9  \
Date                                                                        
2023-09-18 04:00:00+00:00   638300        0.0           0.0           NaN   
2023-09-19 04:00:00+00:00   849000        0.0           0.0           NaN   
2023-09-20 04:00:00+00:00   773500        0.0           0.0           NaN   
2023-09-21 04:00:00+00:00  1078900        0.0           0.0           NaN   
2023-09-22 04:00:00+00:00  1159300        0.0           0.0           NaN   
...                            ...        ...           ...           ...   
2024-03-11 04:00:00+00:00   987300        0.0           0.0     -2.672731   
2024-03-12 04:00:00+00:00  1005600        0.0           0.0     -2.412361   
2024-03-13 04:00:00+00:00   750100        0.0           0.0     -2.227143   
2024-03-14 04:00:00+00:00  1296000        0.0           0.0     -2.420409   
2024-03-15 04:00:00+00:00  2467000        0.0           0.0     -2.596895   

                           MACDh_12_26_9  MACDs_12_26_9       ATR Signal  
Date                                                                      
2023-09-18 04:00:00+00:00            NaN            NaN       NaN      N  
2023-09-19 04:00:00+00:00            NaN            NaN       NaN      N  
2023-09-20 04:00:00+00:00            NaN            NaN       NaN      N  
2023-09-21 04:00:00+00:00            NaN            NaN       NaN      N  
2023-09-22 04:00:00+00:00            NaN            NaN       NaN      N  
...                                  ...            ...       ...    ...  
2024-03-11 04:00:00+00:00       0.033896      -2.706627  6.648849      B  
2024-03-12 04:00:00+00:00       0.235413      -2.647774  6.501056      B  
2024-03-13 04:00:00+00:00       0.336505      -2.563647  6.285957      B  
2024-03-14 04:00:00+00:00       0.114591      -2.535000  6.339824      B  
2024-03-15 04:00:00+00:00      -0.049516      -2.547379  6.306261      S  

[125 rows x 12 columns]

In [44]:
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Candlestick(x=df.index,
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACD_12_26_9'], mode='lines', name='MACD Line'), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df['MACDs_12_26_9'], mode='lines', name='Signal Line'), row=2, col=1)
fig.add_trace(go.Bar(x=df.index, y=df['MACDh_12_26_9'], name='Histogram'), row=2, col=1)

fig.show()


In [53]:
class MyStrategy(Strategy):
  size = 100
  slCoefficient = 1.1
  tpCoefficient = 1.5

  def init(self):
    super().init()
    self.signal = self.data.Signal

  def next(self):
    super().next()
    stopLossAtr = self.slCoefficient * (self.data.ATR[-1] if not np.isnan(self.data.ATR[-1]) else 6)

    if self.signal == 'B' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] - stopLossAtr
      takeProfit = self.data.Close[-1] + self.tpCoefficient * stopLossAtr
      self.buy(sl=stopLoss, tp=takeProfit, size=self.size)

    elif self.signal == 'S' and len(self.trades) == 0:
      stopLoss = self.data.Close[-1] + stopLossAtr
      takeProfit = self.data.Close[-1] - self.tpCoefficient * stopLossAtr
      self.sell(sl=stopLoss, tp=takeProfit, size=self.size)


bt = Backtest(df, MyStrategy, cash=1000000000, commission=.002)
results = bt.run()

# Print the backtesting results
print(results)


Start                     2023-09-18 04:00...
End                       2024-03-15 04:00...
Duration                    179 days 00:00:00
Exposure Time [%]                        98.4
Equity Final [$]                9994637.90102
Equity Peak [$]               10003536.597377
Return [%]                          -0.053621
Buy & Hold Return [%]                 9.00862
Return (Ann.) [%]                    -0.10807
Volatility (Ann.) [%]                 0.06251
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.107326
Avg. Drawdown [%]                   -0.027078
Max. Drawdown Duration      140 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                   26
Win Rate [%]                        30.769231
Best Trade [%]                        3.60687
Worst Trade [%]                     -5.513667
Avg. Trade [%]                    